# Running The Model

### Init

In [ ]:
import os
import hack
from DatasetLoader import *
from DataGenerator import *
import pickle
with open('../Models/Dataset_Simple_scaler.pickle', 'rb') as handle:
    dataset_scaler = pickle.load(handle)

In [ ]:
import os
TEST_PATH='../Test_Results/Temporal-Prop-Network-Fixed'
if os.path.exists(TEST_PATH):
    print(Exception('This directory already exists'));
else:    
    os.mkdir(TEST_PATH);
    os.mkdir(TEST_PATH+'/saved_models')

In [ ]:
## ALT
my_dataset9 = MyDataset(PATH='../Data/DATASET_FIXED_ONLY/9Objects/',n_of_scene=1100,n_of_exp=4,n_of_obj=9,f_size=8,n_of_rel_type=1,fr_size=240,scaler=dataset_scaler)    
my_dataset6 = MyDataset(PATH='../Data/DATASET_FIXED_ONLY/6Objects/',n_of_scene=50,n_of_exp=4,n_of_obj=6,f_size=8,n_of_rel_type=1,fr_size=240,scaler=dataset_scaler)    
my_dataset12 = MyDataset(PATH='../Data/DATASET_FIXED_ONLY/12Objects/',n_of_scene=50,n_of_exp=4,n_of_obj=12,f_size=8,n_of_rel_type=1,fr_size=240,scaler=dataset_scaler)    
my_dataset9.divideDataset(9.0/11,1.5/11)
my_dataset6.divideDataset(0.0,0.5)
my_dataset12.divideDataset(0.0,0.5)

In [ ]:
my_dataset_test6 = MyDataset2(PATH='../Data/DATASET_TEST_Simple/6Objects/',n_of_scene=50,n_of_exp=4,n_of_groups=7,n_of_obj=6,f_size=5,n_of_rel_type=1,fr_size=50,scaler=dataset_scaler)
my_dataset_test6.divideDataset(0,0)
my_dataset_test8 = MyDataset2(PATH='../Data/DATASET_TEST_Simple/8Objects/',n_of_scene=50,n_of_exp=4,n_of_groups=10,n_of_obj=8,f_size=5,n_of_rel_type=1,fr_size=50,scaler=dataset_scaler)
my_dataset_test8.divideDataset(0,0)
my_dataset_test9 = MyDataset2(PATH='../Data/DATASET_TEST_Simple/9Objects/',n_of_scene=50,n_of_exp=4,n_of_groups=12,n_of_obj=9,f_size=5,n_of_rel_type=1,fr_size=50,scaler=dataset_scaler)
my_dataset_test9.divideDataset(0,0)

In [ ]:
import hack
from Networks import *
Pns= PropagationNetwork()
Pn1=Pns.getModel(10,6,1)

In [ ]:
from Test import *

Pns.setModel(10,'../Models/PN_fixed.hdf5')
_=Pns.getModel(7,6,1)
_=Pns.getModel(9,6,1)
_=Pns.getModel(10,6,1)
_=Pns.getModel(13,6,1)

In [ ]:
# TODO: Take this to test.src

def custom_Test(dataset,relations,startStep,Ins,frame_len,relation_threshold):
    n_objects = dataset.n_of_obj+1
    num_of_rel_type=dataset.num_of_rel_type
    if num_of_rel_type>1:
        num_of_rel_type=num_of_rel_type+1
    n_relations=n_objects*(n_objects-1)
    In=Ins.getModel(n_objects)
    GroundData=dataset.data[dataset.test_traj_start:,startStep:,:,:]
    n_of_traj=GroundData.shape[0]
    
    xy_origin_pos=copy.deepcopy(GroundData[:,:frame_len,:,2:4]);
    xy_origin_vel=copy.deepcopy(GroundData[:,:frame_len,:,4:6]);
    
    dataToModel= np.zeros([n_of_traj,frame_len,n_objects,6])
    dataToModel[:,0,:,:]= copy.deepcopy(GroundData[:,0,:,:])
    dataToModel[:,:,0,:]= copy.deepcopy(GroundData[:,:frame_len,0,:])
    dataToModel[:,:,:,:2]= copy.deepcopy(GroundData[:,:frame_len,:,:2])
    dataToModel[:,0,1:,4:6]= 0
    r= dataToModel[:,0,:,0]
    dataToModel=dataset.scaler.transform(dataToModel)
    val_receiver_relations = np.zeros((n_of_traj, n_objects, n_relations), dtype=float);
    val_sender_relations   = np.zeros((n_of_traj, n_objects, n_relations), dtype=float);
    val_relation_info = relations
    propagation = np.zeros((n_of_traj, n_objects, 100))
    cnt = 0
    for m in range(n_objects):
        for j in range(n_objects):
            if(m != j):
                inzz=np.linalg.norm(dataToModel[:,0,m,2:4]-dataToModel[:,0,j,2:4],axis=1)<relation_threshold
                val_receiver_relations[inzz, j, cnt] = 1.0
                val_sender_relations[inzz, m, cnt]   = 1.0
                cnt += 1
    edges=val_relation_info
    for i in range(1,frame_len):
        velocities=In.predict({'objects': dataToModel[:,i-1,:,:],'sender_relations': val_sender_relations,'receiver_relations': val_receiver_relations,'relation_info': val_relation_info,'propagation':propagation})
        dataToModel[:,i,1:,2:4]=dataToModel[:,i-1,1:,2:4]
        dataToModel[:,i,1:,4:6]=velocities[:,:,:]; 
        dataToModel[:,i,1:,:]=PositionCalculateNext(dataToModel[:,i,1:,:],dataset.scaler)
        val_receiver_relations = np.zeros((n_of_traj, n_objects, n_relations), dtype=float);
        val_sender_relations   = np.zeros((n_of_traj, n_objects, n_relations), dtype=float);
        cnt = 0
        for m in range(n_objects):
            for j in range(n_objects):
                if(m != j):
                    inzz=np.linalg.norm(dataToModel[:,i,m,2:4]-dataToModel[:,i,j,2:4],axis=1)<relation_threshold
                    val_receiver_relations[inzz, j, cnt] = 1.0
                    val_sender_relations[inzz, m, cnt]   = 1.0                    
                    cnt += 1
    pred_xy = dataset.scaler.inv_transform(dataToModel)
    
    xy_calculated_pos=pred_xy[:,:,:,2:4]
    xy_calculated_vel=pred_xy[:,:,:,4:6]
    print 'mse-pos:',np.log(mean_squared_error(xy_calculated_pos[:,:,1:,:].reshape(-1,2),xy_origin_pos[:,:,1:,:].reshape(-1,2)))
    print 'mse-vel:',np.log(mean_squared_error(xy_calculated_vel[:,:,1:,:].reshape(-1,2),xy_origin_vel[:,:,1:,:].reshape(-1,2)))
    return mean_squared_error(xy_calculated_pos[:,:,1:,:].reshape(-1,2),xy_origin_pos[:,:,1:,:].reshape(-1,2))

### Running Model on Test Set Sparse

In [ ]:
import hack
from Networks import *
Tpns = TemporalPropagationNetwork()
Tpn = Tpns.getModel(10,6,1,100,50)

In [ ]:
from Test import *
Tpn.load_weights(('../Models/TPN_fixed.hdf5'))
Tpn9 = Tpns.getModel(10,6,1,240,0)
Tpn6 = Tpns.getModel(7,6,1,240,0)
Tpn12 = Tpns.getModel(13,6,1,240,0)

In [ ]:
xy_origin_pos9,edge9,r9,predicted_edges9=Test_TPN(my_dataset9,Tpn9,240,dataset_scaler.relation_threshold)
xy_origin_pos6,edge6,r6,predicted_edges6=Test_TPN(my_dataset6,Tpn6,240,dataset_scaler.relation_threshold)
xy_origin_pos12,edge12,r12,predicted_edges12=Test_TPN(my_dataset12,Tpn12,240,dataset_scaler.relation_threshold)

In [ ]:
predicted_edges_alt=dict()
predicted_edges_alt[9]=predicted_edges9.copy()
predicted_edges_alt[9][predicted_edges_alt[9]>0.7]=1
predicted_edges_alt[9][predicted_edges_alt[9]<0.8]=0

predicted_edges_alt[6]=predicted_edges6.copy()
predicted_edges_alt[6][predicted_edges_alt[6]>0.7]=1
predicted_edges_alt[6][predicted_edges_alt[6]<0.8]=0

predicted_edges_alt[12]=predicted_edges12.copy()
predicted_edges_alt[12][predicted_edges_alt[12]>0.7]=1
predicted_edges_alt[12][predicted_edges_alt[12]<0.8]=0

datasets=dict()
datasets[6]=my_dataset6
datasets[9]=my_dataset9
datasets[12]=my_dataset12

In [ ]:
errors=dict()
for obj in [6,9,12]:
    errors[obj]=list()
for ts in [20,40,60,80,100,120,140,160,180,200]:
    for objects in [6,9,12]:
        errors[objects].append(custom_Test(datasets[objects],predicted_edges_alt[objects][:,:,ts,:],ts,Pns,240-ts,dataset_scaler.relation_threshold))

### Running Model on Test Set Dense

In [ ]:
Tpn6 = Tpns.getModel(7,6,1,50,0)
Tpn8 = Tpns.getModel(9,6,1,50,0)
Tpn9 = Tpns.getModel(10,6,1,50,0)

In [ ]:
xy_origin_pos6,edge6,r6,predicted_edges6=Test_TPN(my_dataset_test6,Tpn6,50,dataset_scaler.relation_threshold)
xy_origin_pos8,edge8,r8,predicted_edges8=Test_TPN(my_dataset_test8,Tpn8,50,dataset_scaler.relation_threshold)
xy_origin_pos9,edge9,r9,predicted_edges9=Test_TPN(my_dataset_test9,Tpn9,50,dataset_scaler.relation_threshold)

In [ ]:
predicted_edges_alt=dict()
predicted_edges_alt[6]=predicted_edges6.copy()
predicted_edges_alt[6][predicted_edges_alt[6]>0.7]=1
predicted_edges_alt[6][predicted_edges_alt[6]<0.8]=0

predicted_edges_alt[8]=predicted_edges8.copy()
predicted_edges_alt[8][predicted_edges_alt[8]>0.7]=1
predicted_edges_alt[8][predicted_edges_alt[8]<0.8]=0

predicted_edges_alt[9]=predicted_edges9.copy()
predicted_edges_alt[9][predicted_edges_alt[9]>0.7]=1
predicted_edges_alt[9][predicted_edges_alt[9]<0.8]=0

datasets=dict()
datasets[6]=my_dataset_test6
datasets[8]=my_dataset_test8
datasets[9]=my_dataset_test9

In [ ]:
errors=dict()
for obj in [6,8,9]:
    errors[obj]=list()
for ts in [5,10,15,20,25,30]:
    for objects in [6,8,9]:
        errors[objects].append(custom_Test(datasets[objects],predicted_edges_alt[objects][:,:,ts,:],ts,Pns,50-ts,dataset_scaler.relation_threshold))